# Chapter 2: Reasoning About Code - A New Way

In Chapter 1, we implemented a working 1-D heat equation solver.
It runs, but it’s fragile, opaque, and hard to test or extend.

In this chapter, we ask: can we design and reason about our code systematically, before we even run it?

We’ll introduce preconditions, postconditions, and invariants (concepts borrowed from formal logic)
and use them to refactor step by step into something more modular, testable, and robust.

---

### Goals

 - Identify preconditions, postconditions, and invariants for a scientific computing problem.
 - Apply stepwise refinement to improve a messy, monolithic implementation.
 - Separate concerns in your code: boundary conditions, flux computations, and updates.
 - Produce a modular version of the heat equation solver that sets the stage for unit and property-based testing.

### Key Concepts

 - Abstraction: Viewing complex systems at a high-level, where irrelevant details are ignored.
 - Hoare Logic: reasoning about code with `{P} code {Q}` triples.
 - Stepwise refinement: incrementally developing code.
 - Invariants: conditions that must always hold true during program execution.
 - Separation of concerns: organizing code into clean, testable components.

---


## Why Reason About Code?

Scientific software is both:
 - A *tool* that enables science,
 - A *research product* in its own right.

As such, it warrants a similar rigor and reasoning as the science it enables. 

Toward the goal of understanding and improving our code, we can even take inspiration from the scientific method:

- Form hypotheses -> attempt refutation → refine toward confidence.

Reasoning About Code:

- Specified properties and behaviors -> Test / Check against specifications -> Refine implementation.

In practice, much scientific software evolves through a *code and fix* process:

> If the outputs look reasonable and it doesn't crash, it must be correct.

But trusting results without understanding the code is risky.

> "We can't get systems right if we don't understand them" - Leslie Lamport


### Abstraction

Abstraction lets us reason about what a system should do without drowning in how it’s done. 

As scientists and engineers, we are already accustomed to this way of thinking: we model complex systems by removing unnecessary details and breaking problems into components.

**Where do the low-level details go?**

**A natural concern:** Aren’t most bugs caused by subtle low-level issues?  

Yes, but many of those "low-level" bugs are really symptoms of unclear high-level intent.
When top-level responsibilities are muddled or poorly defined, they trickle down into brittle,
error-prone implementations.  

Abstraction then pays off in two ways:
 - Reasoning: We can understand and predict behavior at a manageable scale.
 - Design: We can isolate low-level details, reducing complexity and making the system easier to extend.

> The hard part of building software is the conceptual construct, 
> not the labor of representing it.
> *-FP Brooks. “No Silver Bullet” (1987)*

> What matters is the fundamental structure of the design. If you get
> it wrong, there is no amount of bug fixing and refactoring that will
> produce a reliable, maintainable, and usable system
> *- D. Jackson. The essence of software. (2021)*


### Specification: Writing Down Intent

A *specification* is a precise statement of what software should do, independent of how it does it.

Specifications can range from informal descriptions in plain English
to fully formal mathematical models.  In this tutorial, we’ll use a
**lightweight, practical level**: preconditions and postconditions
expressed as simple `assert` statements directly in code.

This lightweight approach:
 - Guides implementation,
 - Facilitates testing and validation.

### Floyd-Hoare Triples

A Hoare triple expresses the contract for a code fragment:

`  {P} code {Q}`

where
 - `P` (precondition): what must be true before running the `code`.
 - `code` (implementation): the program fragment in question, e.g., a statement, block, function, etc.
 - `Q` (postcondition): what must be true after running the `code`.

#### Exercise 2.1:

Can you find inputs where this function fails the postcondition even when the precondition is satisfied?

In [1]:
def div(x, y):
    assert y != 0           # P    (precondition)
    res = x / y             # code (implementation)
    assert res * y == x     # Q    (postcondition)
    return res

In [2]:
div(4, 2)

2.0

*Observation:* Floating-point arithmetic introduces unavoidable rounding errors.
These aren’t bugs, but natural computational artifacts.
We can **weaken the postcondition** to account for this by using approximate equality.

In [3]:
import numpy as np

def div(x, y):
    assert y != 0                   # P    (precondition)
    res = x / y                     # code (implementation)
    assert np.isclose(res * y, x)   # Q    (postcondition)
    return res

In [4]:
div(7, 25)

0.28

**Takeaway**: Postconditions should reflect computational realities (e.g., floating-point).

**Preconditions deserve similar care.**

A **weakest precondition** is the *most general condition* under which the code
works correctly. It ensures your function is usable in the widest range of scenarios
while still preventing incorrect behavior.

Conversely, we want the **strongest postcondition** possible: the most precise guarantee
about the outcome. This balance maximizes both the flexibility and reliability of our code.



## Stepwise Refinement: From Abstract to Concrete

Stepwise refinement is the process of progressively developing a solution, starting with high-level considerations and gradually refining it into an implementation.

The process:
 1. **Identify the What:** Define the overall problem and desired outcome.
 2. **Choose the Abstractions:** Select the conceptual building blocks.
 3. **Design the Core API:** the main interfaces and interactions
 4. **Implement Incrementally:** Fill in implementations step by step.

Benefits:
 - Clear separation of concerns,
 - Local, testable invariants,
 - Flexibility to change or replace components,
 - Robustness that emerges naturally.

## Heat Equation Solver, Revisited

We now apply stepwise refinement to redesign our 1-D heat equation solver.

### Step 1 — Specification (the what)

We want to advance a 1-D temperature field $u_i$ on a uniform grid using a conservative finite volume method:

$
\qquad
u_i^{n+1} = u_i^{n} + \Delta t \, \frac{F_{i} - F_{i+1}}{\Delta x},
\qquad
F_i =
\begin{cases}
q_L & i = 0, \\[6pt]
-\kappa \dfrac{u_i - u_{i-1}}{\Delta x} & 1 \leq i \leq N-1, \\[10pt]
q_R & i = N.
\end{cases}
$

#### Preconditions

- `len(u) == N >= 3`, `len(F) == N+1`, `len(dudt) == N` 
- `dx > 0`, `dt > 0`, `nt > 0`, `kappa > 0`
- Stability condition for explicit Euler (FTCS):
$$
\frac{\kappa \Delta t}{\Delta x^2} \leq \frac{1}{2}
$$

#### Postconditions

- Conservation (per step):
$$
\sum_{i=0}^{N-1} u_i^{n+1} \Delta x = \sum_{i=0}^{N-1} u_i^{n} \Delta x + \Delta t \left( q_L - q_R \right).
$$

- Special Cases:
    - If `q_L` == `q_R` == 0, the mean stays constant.
    - If `q_L` and `q_R` are constant, the temperature profile converges to a linear steady state.

### Step 2 – Choose the abstractions

If we look at the 1-D heat equation solver at an abstract level, we don’t begin with arrays,
loops, or numerical schemes. We begin with the concepts: the fundamental entities in the
world of this problem that the software must represent faithfully and consistently.

Concepts we commit to, upfront, are then:

 - *Mesh*: domain geometry: `(dx, N)`.
 - *BC*: boundary heat fluxes: `(qL, qR)`.
 - *Cell Field*: cell-centered field $u$, length $N$.
 - *Face Field*: face-centered field $F$, length $N+1$.
 - Operations:
    - `apply_bc(F, qL, qR)`
    - `flux(u, kappa, dx)`, 
    - `divergence(F, mesh)`, 
    - `step(...)`, 
    - `solve(...)`.

This gives a small, coherent core that matches the math and isolates concerns.

### Step 3 – Design the Core API

Our task now is to design a small but complete core API: a minimal set of data structures
and functions that capture the essential ideas of our solver. This isn’t about writing code yet:
it’s about defining the conceptual architecture of the system.

The goal is to capture the set of abstractions we have identified, i.e., the fundamental
building blocks of the solver, and relationships among them. These abstractions form the
"language" of the system: every part of the solver will communicate through them.

Below is a core API that encodes the fundamental concepts of our solver.
This API is small, expressive, and built around the essential entities of the problem: 
the mesh, the boundary conditions, and *state transitions* that move our system forward in time.

In [5]:
from dataclasses import dataclass
from typing import List, Sequence

# Data abstractions

@dataclass
class Mesh:
    """Uniform 1-D mesh."""
    dx: float
    N: int  # number of cells
    def cell_centered_arr(self): return [0.0] * self.N
    def face_centered_arr(self): return [0.0] * (self.N + 1)

@dataclass
class BC:
    """Neumann flux boundary conditions on the diffusive flux."""
    qL: float
    qR: float

# Behavior abstractions

def apply_bc(F: list, qL: float, qR: float) -> None:
    """Apply Neumann BCs to face fluxes F."""
    ...

def flux(F: list, u: list, kappa: float, dx: float) -> None:
    """Update interior face fluxes F based on cell field u."""
    ...

def divergence(dudt: list, F: list, dx: float) -> None:
    """Update cell tendencies dudt based on face fluxes F."""
    ...

def step(u: list, F: list, dudt: list, kappa: float, dt: float, mesh: Mesh, bc: BC) -> list:
    """Advance cell field u by one time step using explicit Euler method."""
    ...

def solve(u0: list, kappa: float, dt: float, nt: int, dx: float, bc: BC) -> list:
    """Orchestrate nt steps over cell field u."""
    ...

### Step 4 – Implement Incrementally: Fill in implementations step by step, maintaining the contracts.

 - Step 4a — Contracts first: write the behavioral *facts* of functions (preconditions, postconditions, invariants) without code.
 - Step 4b — Concrete code: implement to those contracts, with lightweight asserts that make the spec executable.


**Why split these steps?** This mirrors science itself: hypotheses come before
experiments, just as specifications come before code. This also makes failures diagnostic:
if a 4a assertion trips, your spec is wrong or incomplete. If 4b trips, your code violates
a correct spec.


#### Step 4a: Contracts

Before diving into coding, it may be worth thinking about what pre- and/or postconditions
we want to hold before, during, and after execution of our functions.

**Preconditions**

Let's begin with a few simple preconditions on the input arguments of our functions. Subsequently, we will implement a precondition that checks the stability condition for the explicit Euler (FTCS) scheme.

In [6]:
def apply_bc(F: list, qL: float, qR: float) -> None:
    """Apply Neumann BCs to face fluxes F."""
    assert len(F) > 1
    ...

def flux(F: list, u: list, kappa: float, dx: float) -> None:
    """Update face fluxes F based on cell field u."""
    assert dx > 0 and len(F) == len(u) + 1
    ...

def divergence(dudt: list, F: list, dx: float) -> None:
    """Update cell tendencies dudt based on face fluxes F."""
    assert len(F) == len(dudt) + 1
    ...

def step(u: list, F: list, dudt: list, kappa: float, dt: float, mesh: Mesh, bc: BC) -> None:
    """Advance cell field u by one time step using explicit Euler method."""
    assert dt > 0

def solve(u0: list, kappa: float, dt: float, nt: int, dx: float, bc: BC) -> list:
    """Orchestrate nt steps over cell field u."""
    assert nt > 0



#### Exercise 2.2: 

One of the key preconditions for the Euler (FTCS) scheme is the stability condition
as given below:

$$\frac{\kappa \Delta t}{\Delta x^2} < \frac{1}{2}$$

Implement a function to check the stability condition. Which function(s) would you modify to incorporate this check as an assertion? 

In [7]:
def Euler_FTCS_stability(dt: float, dx: float, kappa: float) -> bool:
    """Check stability condition for explicit Euler (FTCS) scheme."""
    ...

**Answer:** 

The function `Euler_FTCS_stability` can be used to check the stability condition
as a precondition in the `solve` function. If the condition is not met, we can raise an error
or adjust the time step accordingly.

In [8]:
def Euler_FTCS_stability(dt: float, dx: float, kappa: float) -> bool:
    """Check stability condition for explicit Euler (FTCS) scheme."""
    r = kappa * dt / dx**2
    return r <= 0.5

**Postconditions**

Consider the update rule for cell-centered tendencies:

$$
\text{dudt[i]} = \frac{F_{i-1} - F_i}{\Delta x}
$$

The finite volume telescoping property states that summing the tendencies (`dudt[i]`) over all cells and multiplying by `dx` must equal the net boundary flux (`F[0] - F[-1]`). Below is a function that checks if this property holds, which can be called as a postcondition after each 
time `divergence` is computed:


In [9]:
def divergence_telescoping(F, dudt, dx):
    """Check the finite volume telescoping property."""
    total_divergence = sum(dudt) * dx
    boundary_flux = F[0] - F[-1]
    return np.isclose(total_divergence, boundary_flux)

Below is how this function could be utilized as a postcondition after each time `divergence` is computed:

In [19]:
def divergence(dudt: list, F: list, mesh: Mesh) -> None:
    """Update cell tendencies dudt based on face fluxes F."""
    assert len(dudt) == mesh.N and len(F) == mesh.N + 1 # P (precondition)
    ...                                                 # code
    assert divergence_telescoping(F, dudt, mesh)        # Q (postcondition)

#### Exercise 2.3: 

Now, let's check whether the heat is conserved, a condition that's closely related to the `divergence_telescoping` check.
Recall the condition for heat conservation:

$$
\sum_{i=0}^{N-1} u_i^{n+1} \Delta x = \sum_{i=0}^{N-1} u_i^{n} \Delta x + \Delta t \left( q_L - q_R \right).
$$

Similar to how we checked for divergence, implement a function to check the conservation of 
heat. The function arguments should give you a good hint about how to implement this.
Which function(s) would you modify to incorporate this check?

In [12]:
def heat_is_conserved(u_sum_before: float, u_sum_after: float, dt: float, bc: BC, mesh: Mesh) -> bool:
    """Check if heat is conserved."""
    ...

**Answer**

The following cell contains a possible implementation of the heat conservation check.
As for which function(s) would need to be modified to incorporate this check, it may be
added as a postcondition to the `step` function, or, alternatively, it may be added
as a loop invariant in the timestepping loop within the `solve` function.

In [13]:
def heat_is_conserved(u_sum_before: float, u_sum_after: float, dt: float, bc: BC, mesh: Mesh) -> bool:
    """Check if heat is conserved."""
    expected_change = dt * (bc.qL - bc.qR)
    actual_change = u_sum_after - u_sum_before
    return np.isclose(actual_change, expected_change)

**Loop invariants**

A loop invariant is a condition (a boolean expression) that holds true before and after each iteration of a loop. It helps reason about the correctness of the loop by ensuring that certain properties hold:

  - before the loop starts
  - after each iteration of the loop
  
which guarantees that once the loop has finished executing, the desired property holds for the final state.

In the case of the heat equation solver, ensuring that heat is conserved after each time step guarantees that the total heat within the domain remains constant.

#### Step 4b: Concrete Code

In [16]:
from dataclasses import dataclass
from typing import List, Sequence
import numpy as np

# Data abstractions

@dataclass
class Mesh:
    """Uniform 1-D mesh."""
    dx: float
    N: int  # number of cells
    def cell_centered_arr(self): return [0.0] * self.N
    def face_centered_arr(self): return [0.0] * (self.N + 1)

@dataclass
class BC:
    """Neumann flux boundary conditions on the diffusive flux."""
    qL: float
    qR: float

# Behavior abstractions

def apply_bc(F: list, qL: float, qR: float) -> None:
    """Apply Neumann BCs to face fluxes F."""
    assert len(F) > 1
    F[0], F[-1] = qL, qR

def flux(F: list, u: list, kappa: float, dx: float) -> None:
    """Update face fluxes F based on cell field u."""
    assert dx > 0 and len(F) == len(u) + 1
    for i in range(1, len(u)):
        F[i] = -kappa * (u[i] - u[i-1]) / dx

def divergence(dudt: list, F: list, dx: float) -> None:
    """Update cell tendencies dudt based on face fluxes F."""
    assert len(F) == len(dudt) + 1
    for i in range(len(dudt)):
        dudt[i] = (F[i] - F[i+1]) / dx
    assert divergence_telescoping(F, dudt, dx)

def step(u: list, F: list, dudt: list, kappa: float, dt: float, mesh: Mesh, bc: BC) -> list:
    """Advance cell field u by one time step using explicit Euler method."""
    assert dt > 0

    apply_bc(F, bc.qL, bc.qR)
    flux(F, u, kappa, mesh.dx)
    divergence(dudt, F, mesh.dx)

    for i in range(mesh.N):
        u[i] += dt * dudt[i]

def solve(u0: list, kappa: float, dt: float, nt: int, dx: float, bc: BC) -> list:
    """Orchestrate nt steps over cell field u."""

    assert nt > 0
    assert Euler_FTCS_stability(dt, dx, kappa), "Stability condition not met"

    mesh = Mesh(dx, N=len(u0))
    u = u0.copy()
    F = mesh.face_centered_arr()
    dudt = mesh.cell_centered_arr()

    for _ in range(nt):
        u_sum_before = sum(u) * mesh.dx
        step(u, F, dudt, kappa, dt, mesh, bc)
        u_sum_after = sum(u) * mesh.dx
        assert heat_is_conserved(u_sum_before, u_sum_after, dt, bc, mesh), "Heat not conserved"

    return u


In [17]:
solve(
    u0 = [0.0, 100.0, 0.0],
    kappa = 0.1,
    dt = 1.0,
    nt = 1000,
    dx = 1.0,
    bc = BC(qL=0.0, qR=0.0)
)

[33.333333333333314, 33.33333333333333, 33.333333333333314]

## What we just did

We now have a solver whose structure and reasoning are explicit. It’s modular: each function 
has a single purpose over well-defined data.

We did write a lot more lines, because we added:
 - Separation of concerns (apply_bc, flux, divergence, step, solve) instead of one monolith
 - Executable contracts (pre/postconditions, invariants)
 - Lightweight data types (Mesh, BC)

**In real applications**, this "ceremony" is a small fraction of total code: The heavy lifting
lives in numerical computations, I/O, parallelization, physics, etc.. The scaffolding pays
off by reducing change amplification and cognitive load:

 - Safer changes (localized edits)
 - Better testing (unit, property, etc.)
 - Faster debugging (failures point to where)
 - Reusability (shared abstractions)

**Do assertions hurt performance?** Potentially, if run every step. In practice you can disable
them for production (`python -O`) or sample them periodically during long runs.

## Looking Ahead

In **Chapter 3**, we’ll take the next step:  
turning these specifications into unit tests that run automatically,
giving us rapid feedback and confidence as our code evolves.


---

*This notebook is a part of the "Rigor and Reasoning in Research Software" (R3Sw) tutorial, led by Alper Altuntas and sponsored by the Better Scientific Software (BSSw) Fellowship Program. Copyright © 2025*